In [1]:
import sqlalchemy as alch

In [2]:
alch.__version__

'2.0.20'

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine, Integer, String, Column, Float, ForeignKey
from sqlalchemy.orm import declarative_base

database_name = 'CoinMarketCap'

# Create the connection URL
user = 'root'
password = '123456789'
host = 'localhost'
port = '3306'
database_url = f'mysql://{user}:{password}@{host}:{port}/{database_name}'

# Create the engine and connect to the database
engine = create_engine(database_url)
connection = engine.connect()

# Test the connection
if connection is not None:
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


In [5]:
# Define the base class for declarative models
Base = declarative_base()

# Define the model:

class Coin(Base):
    __tablename__ = 'coin'
    
    symbol = Column(String(255))
    id = Column(Integer, primary_key=True)


class Prop(Base):
    __tablename__ = 'prop'
    
    coin_id = Column(Integer, primary_key=True)
    
    rank = Column(Integer)

    name = Column(String(255))
    
    low_alltime = Column(Float)
    high_alltime = Column(Float)
    total_supply = Column(Float)
    circulating_supply = Column(Float)
    

class Link(Base):
    __tablename__ = 'link'
    
    coin_id = Column(Integer, primary_key=True)
    
    main = Column(String(255))
    github = Column(String(255))
    historical = Column(String(255))
    

class Tag(Base):
    __tablename__ = 'tag'
    
    id = Column(Integer, primary_key=True)
    
    name = Column(String(255))
    

class CoinTag(Base):
    __tablename__ = 'cointag'
    
    id = Column(Integer, primary_key=True)
    
    coin_id = Column(Integer, ForeignKey("coin.id"))
    tag_id = Column(Integer, ForeignKey("tag.id"))
    

class CoinHistory(Base):
    __tablename__ = 'coinhistory'
    
    id = Column(Integer, primary_key=True)
    
    coin_id = Column(Integer, ForeignKey("coin.id"))
    history_id = Column(Integer, ForeignKey("history.id"))
    

class History(Base):
    __tablename__ = 'history'
    
    id = Column(Integer, primary_key=True)
    
    name = Column(String(255))
    
    time_stamp = Column(String(255))
    
    time_high = Column(String(255))
    time_low = Column(String(255))
    
    open = Column(Float)
    close = Column(Float)
    
    high = Column(Float)
    low = Column(Float)
    
    market_cap = Column(Float)
    volume = Column(Float)
    
    
    
#read files
mainTable = pd.read_csv('table_data.csv')
coinProp = pd.read_csv('coinProp.csv')
links = pd.read_csv('historical_links.csv', names=['name', 'historical'])

#coin table
coin = mainTable[['Symbol', 'Rank']].copy()
coin['Rank'] = coin['Rank'].str.split("'").str[1]
coin['Symbol'] = coin['Symbol'].str.split("'").str[1]
coin.columns = ['symbol', 'id']

#prop table
prop = coinProp[['Name', 'Circulating Supply', 'Total Supply', 'Low All Time', 'High All Time']].copy()
prop['id'] = coin['id'].copy()
prop['rank'] = coin['id'].copy()
prop.columns = ['name', 'circulating_supply', 'total_supply', 'low_alltime', 'high_alltime', 'coin_id', 'rank']
prop['name'] = prop['name'].str.split("'").str[1]
order = ['coin_id', 'rank', 'name', 'low_alltime', 'high_alltime', 'total_supply', 'circulating_supply']
prop = prop.reindex(columns=order)

#link table
link = links[['historical']].copy()
link['github'] = coinProp['Source GitHub']
link['coin_id'] = coin['id'].copy()
link['main'] = mainTable['Name'].copy().str.split("'").str[3]
order = ['coin_id', 'main', 'github', 'historical']
link = link.reindex(columns=order)

#tag table
tag = coinProp[['Tags']].copy()
tag['Tag_Names'] = tag['Tags'].apply(lambda x: x.strip("[]").replace("'", "").split(", "))
tag = tag.explode('Tag_Names')
tag = tag[['Tag_Names']].copy()
tag = tag.drop_duplicates()
tag.reset_index(inplace=True)
tag = tag[['Tag_Names']]
tag.columns = ['name']
tag['id'] = range(1, len(tag) + 1)
order = ['id', 'name']
tag = tag.reindex(columns=order)


#cointag table______________________________________________________
tagtmp = coinProp[['Tags']].copy()
tagtmp.columns = ['tags']
tagtmp['coin_id'] = coin['id'].copy()
cointag = pd.DataFrame(columns=['coin_id', 'tag_id'])
# Iterate over each row in the 'tag' dataframe
for index, row in tag.iterrows():
    # Extract the tag value
    tag_value = row['name']  
    # Check if the 'tags' column in 'tagtmp' dataframe contains the tag value as a substring
    matches = tagtmp[tagtmp['tags'].str.contains(tag_value)]   
    # Iterate over each matching row in 'tagtmp' dataframe and create a new row in the result dataframe
    for _, match_row in matches.iterrows():
        new_row = pd.DataFrame({'coin_id': [match_row['coin_id']], 'tag_id': [row['id']]})
        cointag = pd.concat([cointag, new_row], ignore_index=True)
#add id        
cointag['id'] = range(1, len(cointag) + 1)
order = ['id', 'coin_id', 'tag_id']
cointag = cointag.reindex(columns=order)
#____________________________________________________________________


#history table_______________________________________________________

# Get the current directory
current_directory = os.getcwd()
# Set the folder path
folder_path = os.path.join(current_directory, "top200")
# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]
# Create an empty dataframe to store the data
history = pd.DataFrame()

# Loop through each CSV file and load it into the dataframe
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=";")
    history = pd.concat([history, df], ignore_index=True)
    
history = history.drop(['timeOpen', 'timeClose'], axis=1)
history['id'] = range(1, len(history) + 1)
history.columns = ['name', 'time_high', 'time_low', 'open', 'high', 'low', 'close', 'volume', 'market_cap', 'time_stamp', 'id']
order = ['id', 'name', 'time_stamp', 'time_high', 'time_low', 'open', 'close', 'high', 'low', 'market_cap', 'volume']
history = history.reindex(columns=order)
#_____________________________________________________________________

#coinhistory table____________________________________________________

coinhistory = pd.DataFrame()
# Iterate over each row in the 'history' dataframe
for index, row in history.iterrows():
    # Get the value of 'name' from the current row
    name = row['name']
    
    # Find the corresponding 'coin_id' in the 'prop' dataframe
    coin_id = prop.loc[prop['name'] == name, 'coin_id'].values[0]
    
    # Create a new dataframe with 'coin_id' and 'id' values
    df = pd.DataFrame({'coin_id': [coin_id], 'history_id': [row['id']]})
    
    # Concatenate the new dataframe with 'coinhistory'
    coinhistory = pd.concat([coinhistory, df], ignore_index=True)

coinhistory['id'] = range(1, len(coinhistory) + 1)
order = ['id', 'coin_id', 'history_id']
coinhistory = coinhistory.reindex(columns=order)
#_____________________________________________________________________

# Create the tables in the database
Base.metadata.create_all(engine)

# Insert the data into the tables
coinhistory.to_sql('coinhistory', con=connection, if_exists='replace', index=False)
history.to_sql('history', con=connection, if_exists='replace', index=False)
cointag.to_sql('cointag', con=connection, if_exists='replace', index=False)
tag.to_sql('tag', con=connection, if_exists='replace', index=False)
coin.to_sql('coin', con=connection, if_exists='replace', index=False)
prop.to_sql('prop', con=connection, if_exists='replace', index=False)
link.to_sql('link', con=connection, if_exists='replace', index=False)

200